In [3]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import Image, ImageTk
import threading
import os
import time
from sklearn.model_selection import train_test_split

class DrowsinessDetectionApp:
    def __init__(self):
        # Initialize main window
        self.root = tk.Tk()
        self.root.title("Drowsiness Detection System")
        self.root.geometry("900x700")
        
        # Variables
        self.running = False
        self.cap = None
        self.drowsy_count = 0
        self.alert_threshold = 5
        self.model = None
        self.DATA_DIR = r"D:\Users\Home_Desktop\Desktop\Mj\PROJECTS\Drowsiness-Detection-Essemble-Learning\eye_dataset"
        
        # Load Haar cascades
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        self.eye_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_eye.xml'
        )
        
        # UI Frames
        self.video_frame = ttk.Frame(self.root)
        self.video_frame.pack(pady=10)
        
        self.control_frame = ttk.LabelFrame(self.root, text="Controls")
        self.control_frame.pack(pady=10, padx=10, fill="x")
        
        self.status_frame = ttk.LabelFrame(self.root, text="Status")
        self.status_frame.pack(pady=10, padx=10, fill="x")
        
        # Video display
        self.video_label = ttk.Label(self.video_frame)
        self.video_label.pack()
        
        # Control elements
        self.train_button = ttk.Button(self.control_frame, text="Train Model", 
                                     command=self.train_model)
        self.train_button.pack(side=tk.LEFT, padx=5)
        
        self.start_button = ttk.Button(self.control_frame, text="Start Detection", 
                                     command=self.start_detection, state='disabled')
        self.start_button.pack(side=tk.LEFT, padx=5)
        
        self.stop_button = ttk.Button(self.control_frame, text="Stop Detection", 
                                    command=self.stop_detection, state='disabled')
        self.stop_button.pack(side=tk.LEFT, padx=5)
        
        # Status elements
        self.status_label = ttk.Label(self.status_frame, text="Status: Inactive")
        self.status_label.pack(pady=5)
        
        self.model_status = ttk.Label(self.status_frame, text="Model: Not Loaded")
        self.model_status.pack(pady=5)
        
        self.accuracy_label = ttk.Label(self.status_frame, text="Accuracy: N/A")
        self.accuracy_label.pack(pady=5)
        
    def load_dataset(self, data_dir):
        images = []
        labels = []
        for state in ['open', 'closed']:
            path = os.path.join(data_dir, state)
            class_num = 0 if state == 'closed' else 1
            for img_name in os.listdir(path):
                try:
                    img_path = os.path.join(path, img_name)
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (24, 24))
                    img = img.astype('float32') / 255.0
                    images.append(img)
                    labels.append(class_num)
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")
        images = np.array(images).reshape(-1, 24, 24, 1)
        labels = np.array(labels)
        return images, labels

    def create_eye_model(self):
        model = models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=(24, 24, 1)),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.Flatten(),
            layers.Dense(64, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])
        return model

    def train_model(self):
        self.train_button.config(state='disabled')
        try:
            # Load dataset
            images, labels = self.load_dataset(self.DATA_DIR)
            X_train, X_test, y_train, y_test = train_test_split(
                images, labels, test_size=0.2, random_state=42
            )
            
            # Train model
            self.model = self.create_eye_model()
            history = self.model.fit(
                X_train, y_train,
                epochs=10,
                batch_size=32,
                validation_data=(X_test, y_test),
                verbose=1
            )
            
            # Evaluate and update UI
            test_loss, test_accuracy = self.model.evaluate(X_test, y_test)
            self.model.save('eye_state_model.h5')
            self.model_status.config(text="Model: Loaded")
            self.accuracy_label.config(text=f"Accuracy: {test_accuracy:.4f}")
            self.start_button.config(state='normal')
            messagebox.showinfo("Success", "Model trained and saved successfully!")
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to train model: {str(e)}")
        finally:
            self.train_button.config(state='normal')

    def preprocess_eye(self, eye_img):
        eye_img = cv2.resize(eye_img, (24, 24))
        eye_img = eye_img.astype('float32') / 255.0
        return np.expand_dims(eye_img, axis=0)

    def detect_drowsiness(self):
        while self.running:
            ret, frame = self.cap.read()
            if not ret:
                break
                
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)
            drowsy_detected = False
            
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                roi_gray = gray[y:y+h, x:x+w]
                eyes = self.eye_cascade.detectMultiScale(roi_gray)
                
                eyes_closed = 0
                for (ex, ey, ew, eh) in eyes:
                    eye = roi_gray[ey:ey+eh, ex:ex+ew]
                    if self.model:
                        processed_eye = self.preprocess_eye(eye)
                        prediction = self.model.predict(processed_eye, verbose=0)
                        if prediction[0][0] > 0.5:  # Closed eye
                            eyes_closed += 1
                            cv2.rectangle(frame, (x+ex, y+ey), 
                                        (x+ex+ew, y+ey+eh), (0, 0, 255), 2)
                        else:
                            cv2.rectangle(frame, (x+ex, y+ey), 
                                        (x+ex+ew, y+ey+eh), (0, 255, 0), 2)
                
                if len(eyes) == 2 and eyes_closed == 2:
                    self.drowsy_count += 1
                    if self.drowsy_count >= self.alert_threshold:
                        drowsy_detected = True
                        cv2.putText(frame, "DROWSY ALERT!", (x, y-10),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                else:
                    self.drowsy_count = 0
            
            # Update UI
            status = "Drowsy Detected!" if drowsy_detected else "Alert"
            self.status_label.config(text=f"Status: {status}")
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame_rgb)
            imgtk = ImageTk.PhotoImage(image=img)
            self.video_label.imgtk = imgtk
            self.video_label.configure(image=imgtk)
            
            time.sleep(0.033)

        self.cap.release()
        self.status_label.config(text="Status: Inactive")

    def start_detection(self):
        if not self.model:
            messagebox.showwarning("Warning", "Please train the model first!")
            return
        self.running = True
        self.cap = cv2.VideoCapture(0)
        self.start_button.config(state='disabled')
        self.stop_button.config(state='normal')
        self.train_button.config(state='disabled')
        self.detection_thread = threading.Thread(target=self.detect_drowsiness)
        self.detection_thread.start()

    def stop_detection(self):
        self.running = False
        self.start_button.config(state='normal')
        self.stop_button.config(state='disabled')
        self.train_button.config(state='normal')

    def run(self):
        self.root.mainloop()

    def __del__(self):
        if self.cap is not None:
            self.cap.release()
        self.root.quit()

if __name__ == "__main__":
    app = DrowsinessDetectionApp()
    app.run()

Epoch 1/10
1/1 [==============================] - 1s 917ms/step - loss: 0.6869 - accuracy: 0.6875 - val_loss: 0.6975 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 33ms/step - loss: 0.6851 - accuracy: 0.4375 - val_loss: 0.6997 - val_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 33ms/step - loss: 0.6859 - accuracy: 0.5000 - val_loss: 0.7007 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 17ms/step - loss: 0.6641 - accuracy: 0.8125 - val_loss: 0.7040 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 34ms/step - loss: 0.7100 - accuracy: 0.3750 - val_loss: 0.7062 - val_accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 33ms/step - loss: 0.6770 - accuracy: 0.6250 - val_loss: 0.7095 - val_accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 17ms/step - loss: 0.7294 - accuracy: 0.5000 - val_loss: 0.7131 - val_accuracy: 0.5000
Epoch 8/10
1/1 [======